# WeatherPy

---

## Starter Code to Generate Random Geographic Coordinates and a List of Cities

In [2]:
pip install citipy


  Using cached citipy-0.0.6.tar.gz (675 kB)
  Preparing metadata (setup.py) ... done
  Using cached kdtree-0.16-py2.py3-none-any.whl (7.7 kB)
  Created wheel for citipy: filename=citipy-0.0.6-py3-none-any.whl size=675969 sha256=c6718b5285a611d75bb810c4ddc60ba0a3405dbadffa86677f6d8bebbdaa91da
  Stored in directory: /Users/ant/Library/Caches/pip/wheels/41/82/37/8bb56372b0d828d226b83e0117780ecbe9f409100ac52364bd
Successfully built citipy
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Impor the OpenWeatherMap API key
from api_keys import weather_api_key

# Import citipy to determine the cities based on latitude and longitude
from citipy import citipy

### Generate the Cities List by Using the `citipy` Library

In [2]:
# Empty list for holding the latitude and longitude combinations
lat_lngs = []

# Empty list for holding the cities names
cities = []

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
print(f"Number of cities in the list: {len(cities)}")

Number of cities in the list: 563


---

## Requirement 1: Create Plots to Showcase the Relationship Between Weather Variables and Latitude

### Use the OpenWeatherMap API to retrieve weather data from the cities list generated in the started code

In [3]:
# Set the API base URL
url = 'https://api.openweathermap.org/data/2.5/weather?'
metric = 'metric'

# Define an empty list to fetch the weather data for each city
city_data = []

# Print to logger
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters
record_count = 1
set_count = 1

# Loop through all the cities in our list to fetch weather data
for i, city in enumerate(cities):
        
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 0

    # Create endpoint URL with each city
    city_url = f"{url}q={city}&appid={weather_api_key}&units={metric}"
    
    # Log the url, record, and set numbers
    print("Processing Record %s of Set %s | %s" % (record_count, set_count, city))

    # Add 1 to the record count
    record_count += 1

    # Run an API request for each of the cities
    try:
        # Parse the JSON and retrieve data
        city_weather = requests.get(city_url).json()

        # Parse out latitude, longitude, max temp, humidity, cloudiness, wind speed, country, and date
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_date = city_weather["dt"]

        # Append the City information into city_data list
        city_data.append({"City": city, 
                          "Lat": city_lat, 
                          "Lng": city_lng, 
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

    # If an error is experienced, skip the city
    except:
        print("City not found. Skipping...")
        pass
              
# Indicate that Data Loading is complete 
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")


Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | as sulayyil
Processing Record 2 of Set 1 | adamstown
Processing Record 3 of Set 1 | utrik
Processing Record 4 of Set 1 | vorgashor
Processing Record 5 of Set 1 | alo
Processing Record 6 of Set 1 | port-aux-francais
Processing Record 7 of Set 1 | ilulissat
Processing Record 8 of Set 1 | dudinka
Processing Record 9 of Set 1 | cabo san lucas
Processing Record 10 of Set 1 | waitangi
Processing Record 11 of Set 1 | ola
Processing Record 12 of Set 1 | las vegas
Processing Record 13 of Set 1 | blackmans bay
Processing Record 14 of Set 1 | atafu village
Processing Record 15 of Set 1 | san juan
Processing Record 16 of Set 1 | kiruhura
Processing Record 17 of Set 1 | blackwater
Processing Record 18 of Set 1 | uvinza
Processing Record 19 of Set 1 | lerwick
Processing Record 20 of Set 1 | quellon
Processing Record 21 of Set 1 | puerto natales
Processing Record 22 of Set 1 | brisas de zicatela
Processing Reco

In [14]:
# Convert the cities weather data into a Pandas DataFrame
city_data_df = pd.DataFrame(city_data)

# Show Record Count
city_data_df.count()

City          533
Lat           533
Lng           533
Max Temp      533
Humidity      533
Cloudiness    533
Wind Speed    533
Country       533
Date          533
dtype: int64

In [16]:
# Display sample data
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,as sulayyil,20.4607,45.5779,31.14,16,0,3.39,SA,1695848557
1,adamstown,-25.0660,-130.1015,20.96,64,96,3.30,PN,1695848840
2,utrik,11.2278,169.8474,28.55,76,98,5.78,MH,1695848840
3,vorgashor,67.5833,63.9500,0.12,84,100,1.20,RU,1695848841
4,alo,7.4178,6.7371,26.16,88,96,1.18,NG,1695848841


In [17]:
# Export the City_Data into a csv
city_data_df.to_csv("output_data/cities.csv", index_label="City_ID")

OSError: Cannot save file into a non-existent directory: 'output_data'

In [12]:
# Read saved data
city_data_df = pd.read_csv("output_data/cities.csv", index_col="City_ID")

# Display sample data
city_data_df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'output_data/cities.csv'

### Create the Scatter Plots Requested

#### Latitude Vs. Temperature

In [ ]:
# Build scatter plot for latitude vs. temperature
x = city_data_df['Lat']
y = city_data_df['Max Temp']

plt.scatter(x,y, marker='o', edgecolors='black')

# Incorporate the other graph properties

plt.title("City Max Latitude vs. Temperature (2023-08-02)")
plt.xlabel("Latitude")
plt.ylabel("Max Temp (C)")
plt.grid(True)
# Save the figure
plt.savefig("output_data/Fig1.png")

# Show plot
plt.show()

#### Latitude Vs. Humidity

In [ ]:
# Build the scatter plots for latitude vs. humidity
plt.scatter(city_data_df['Lat'], city_data_df['Humidity'], marker='o', edgecolors='black')
# Incorporate the other graph properties
plt.title('City Latitude vs. Humidity (2023-08-02)')
plt.xlabel('Latitude')
plt.ylabel('Humidity (%)')
plt.grid(True)

# Save the figure
plt.savefig("output_data/Fig2.png")

# Show plot
plt.show()

#### Latitude Vs. Cloudiness

In [ ]:
# Build the scatter plots for latitude vs. cloudiness
plt.scatter(city_data_df['Lat'], city_data_df['Cloudiness'], marker='o', edgecolors='black')
# Incorporate the other graph properties
plt.title('City Latitude vs. Humidity (2023-08-02)')
plt.xlabel('Latitude')
plt.ylabel('Cloudiness (%)')
plt.grid(True)
# Save the figure
plt.savefig("output_data/Fig3.png")

# Show plot
plt.show()

#### Latitude vs. Wind Speed Plot

In [ ]:
# Build the scatter plots for latitude vs. wind speed
plt.scatter(city_data_df['Lat'], city_data_df['Wind Speed'], marker='o', edgecolors='black')
# Incorporate the other graph properties
plt.title('City Latitude vs. Humidity (2023-08-02)')
plt.xlabel('Latitude')
plt.ylabel('Wind Speed (m/s)')
plt.grid(True)
# Save the figure
plt.savefig("output_data/Fig4.png")

# Show plot
plt.show()

---

## Requirement 2: Compute Linear Regression for Each Relationship


In [ ]:
# Create a DataFrame with the Northern Hemisphere data (Latitude >= 0)
northern_hemi_df = city_data_df.drop(city_data_df[city_data_df["Lat"]<=0].index)

# Display sample data
northern_hemi_df.head()

In [ ]:
# Create a DataFrame with the Southern Hemisphere data (Latitude < 0)
southern_hemi_df = city_data_df.drop(city_data_df[city_data_df["Lat"]>=0].index)

# Display sample data
southern_hemi_df.head()

###  Temperature vs. Latitude Linear Regression Plot

In [ ]:
# Linear regression on Northern Hemisphere
x_values = northern_hemi_df['Lat']
y_values = northern_hemi_df['Max Temp']

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept 
line_eq =  "y = " +str(round(slope,2)) + "x+ " + str(round(intercept,2))

plt.scatter(x_values, y_values, marker='o')
plt.plot(x_values, regress_values, "r-")

plt.title("Northern: Temp vs Lat")
plt.xlabel('Latitude')
plt.ylabel('Max Temp')


plt.annotate(line_eq, (0,0), fontsize = 16, color="red")
print(f"The r-value is: {rvalue}")

plt.show()

In [ ]:
# Linear regression on Southern Hemisphere
x_values = southern_hemi_df['Lat']
y_values = southern_hemi_df['Max Temp']

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept 
line_eq =  "y = " +str(round(slope,2)) + "x+ " + str(round(intercept,2))

plt.scatter(x_values, y_values, marker='o')
plt.plot(x_values, regress_values, "r-")

plt.title("Southern: Temp vs Lat")
plt.xlabel("Latitude")
plt.ylabel("Max Temp")
plt.annotate(line_eq, (-30,0), fontsize = 16, color="red")
print(f"The r-value is: {rvalue}")

plt.show()

**Discussion about the linear relationship:** 
the closer you get to the equater from the the south pole the warmer it gets, andthe same goes for the noth pole towards the equater. 

### Humidity vs. Latitude Linear Regression Plot

In [ ]:
# Northern Hemisphere
x_values = northern_hemi_df['Lat']
y_values = northern_hemi_df['Humidity']

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept 
line_eq =  "y = " +str(round(slope,2)) + "x+ " + str(round(intercept,2))

plt.scatter(x_values, y_values, marker='o')
plt.plot(x_values, regress_values, "r-")
plt.title("Northen Hemi: Humidity vs Latitude")
plt.xlabel('Latitude')
plt.ylabel('Humidity')
plt.annotate(line_eq, (50,15), fontsize = 16, color="red")
print(f"The r-value is: {rvalue}")

plt.show()

In [ ]:
# Southern Hemisphere
x_values = southern_hemi_df['Lat']
y_values = southern_hemi_df['Humidity']

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept 
line_eq =  "y = " +str(round(slope,2)) + "x+ " + str(round(intercept,2))

plt.scatter(x_values, y_values, marker='o')
plt.plot(x_values, regress_values, "r-")
plt.title("Southern Hemi: Humidity vs Latitude")
plt.xlabel('Latitude')
plt.ylabel('Humidity')
plt.annotate(line_eq, (-50,20), fontsize = 16, color="red")
print(f"The r-value is: {rvalue}")

plt.show()

**Discussion about the linear relationship:** their seems to be some sort of correlation in the norhtern hemisphere that the higher the latitude their seems to be more humidity. however for the southern 

### Cloudiness vs. Latitude Linear Regression Plot

In [ ]:
# Northern Hemisphere
# Northern Hemisphere
x_values = northern_hemi_df['Lat']
y_values = northern_hemi_df['Cloudiness']

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept 
line_eq =  "y = " +str(round(slope,2)) + "x+ " + str(round(intercept,2))

plt.scatter(x_values, y_values, marker='o')
plt.plot(x_values, regress_values, "r-")
plt.title("Northen Hemi: Cloudiness vs Latitude")
plt.xlabel('Latitude')
plt.ylabel('Cloudiness')
plt.annotate(line_eq, (50,15), fontsize = 16, color="red")
print(f"The r-value is: {rvalue}")

plt.show()

In [ ]:
# Southern Hemisphere
# Southern Hemisphere
x_values = southern_hemi_df['Lat']
y_values = southern_hemi_df['Cloudiness']

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept 
line_eq =  "y = " +str(round(slope,2)) + "x+ " + str(round(intercept,2))

plt.scatter(x_values, y_values, marker='o')
plt.plot(x_values, regress_values, "r-")
plt.title("Southern Hemi: Cloudiness vs Latitude")
plt.xlabel('Latitude')
plt.ylabel('Cloudiness')
plt.annotate(line_eq, (-50,20), fontsize = 16, color="red")
print(f"The r-value is: {rvalue}")

plt.show()

**Discussion about the linear relationship:** YOUR RESPONSE HERE

### Wind Speed vs. Latitude Linear Regression Plot

In [ ]:
# Northern Hemisphere
x_values = northern_hemi_df['Lat']
y_values = northern_hemi_df['Wind Speed']

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept 
line_eq =  "y = " +str(round(slope,2)) + "x+ " + str(round(intercept,2))

plt.scatter(x_values, y_values, marker='o')
plt.plot(x_values, regress_values, "r-")

plt.title("Northern: Wind Speed vs Lat")
plt.xlabel('Latitude')
plt.ylabel('Wind Speed')
plt.annotate(line_eq, (50,15), fontsize = 16, color="red")
print(f"The r-value is: {rvalue}")

plt.show()

In [ ]:
# Southern Hemisphere
x_values = southern_hemi_df['Lat']
y_values = southern_hemi_df['Wind Speed']

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept 
line_eq =  "y = " +str(round(slope,2)) + "x+ " + str(round(intercept,2))

plt.scatter(x_values, y_values, marker='o')
plt.plot(x_values, regress_values, "r-")

plt.title("Southern: Wind Speed vs Lat")
plt.xlabel('Latitude')
plt.ylabel('Wind Speed')
plt.annotate(line_eq, (-50,8), fontsize = 16, color="red")
print(f"The r-value is: {rvalue}")

plt.show()

**Discussion about the linear relationship:** YOUR RESPONSE HERE